<a href="https://colab.research.google.com/github/daniel-akm/Loan_Class_App/blob/main/Flower_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [4]:
class Model(nn.Module):
  def __init__(self, input_features=4, h1=8, h2=9, output_features=3):
    super().__init__()
    # fc stands for fully connected. do it for each layer. Moves from input_features to h1
    self.fc1 = nn.Linear(input_features, h1)
    # Moves from h1 to h2
    self.fc2 = nn.Linear(h1, h2)
    # Moves for h2 to output_features
    self.out = nn.Linear(h2, output_features)

  # Forward function
  def forward(self, x):
    # Push into first layer
    x = F.relu(self.fc1(x))
    # Push into second layer
    x = F.relu(self.fc2(x))
    # Push into output
    x = self.out(x)
    return x

In [5]:
model = Model()

In [6]:
# import data
import pandas as pd
import numpy as np
# Display chart
import matplotlib.pyplot as plt
%matplotlib inline

In [7]:
url = 'https://gist.githubusercontent.com/netj/8836201/raw/6f9306ad21398ea43cba4f7d537619d0e07d5ae3/iris.csv'
# Dataframe
my_df = pd.read_csv(url)
# Display the first 5
my_df.head()

,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,Setosa
1,4.9,3.0,1.4,0.2,Setosa
2,4.7,3.2,1.3,0.2,Setosa
3,4.6,3.1,1.5,0.2,Setosa
4,5.0,3.6,1.4,0.2,Setosa


In [8]:
# Change the last column from strings to integers
my_df['variety'] = my_df['variety'].replace({'Setosa':0, 'Versicolor':1, 'Virginica':2})
my_df.head()

<ipython-input-8-a8827de58e1a>:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  my_df['variety'] = my_df['variety'].replace({'Setosa':0, 'Versicolor':1, 'Virginica':2})


,sepal.length,sepal.width,petal.length,petal.width,variety
0,5.1,3.5,1.4,0.2,0
1,4.9,3.0,1.4,0.2,0
2,4.7,3.2,1.3,0.2,0
3,4.6,3.1,1.5,0.2,0
4,5.0,3.6,1.4,0.2,0


In [9]:
# Set x (features), y (outcome)
# Remove the last column as it's not a feature
X = my_df.drop('variety', axis=1)
y = my_df['variety']

In [10]:
#  Convert these to numpy arrays
X = X.values
y = y.values

In [11]:
from sklearn.model_selection import train_test_split

In [12]:
# Train Test Split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=41)

In [13]:
# Convert x features to tensors
# Float value because flower features are floats
X_train = torch.FloatTensor(X_train)
X_test = torch.FloatTensor(X_test)

In [14]:
# Convert y labels to tensors
# Long tensors are 64bit integers
y_train = torch.LongTensor(y_train)
y_test = torch.LongTensor(y_test)

In [15]:
#  Set criterion of model to measure error, how far off the predictions are from the data
criterion = nn.CrossEntropyLoss()
# Choose Adam Optimizer, lr = learning rate (if error doesn't go down after a bunch of iterations (epochs), lower our learning rate)
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)
model.parameters

<bound method Module.parameters of Model(
  (fc1): Linear(in_features=4, out_features=8, bias=True)
  (fc2): Linear(in_features=8, out_features=9, bias=True)
  (out): Linear(in_features=9, out_features=3, bias=True)
)>

In [16]:
# HYPER-PARAMETERS
EPOCHS = 20

In [17]:
# Train Model
def train_model(model, data, optimizer, loss_fn, epochs):
  # Epochs
  for i in range(epochs):
    # Keep track of the total_loss
    total_loss=0
    for X, y in data:
      # Predicted values for each X_train data
      pred_val = model(X)
      # Calculate the error between pred_val and actual_data using BinaryCrossEntropy
      loss = loss_fn(pred_val, y)
      # Update the total loss
      total_loss += loss.item()

      # Clear the gradients
      optimizer.zero_grad()
      # Backpropagation
      loss.backward()
      # Step the optimizer by the learning rate
      optimizer.step()

    # Print epochs and losses
    print(f'Epoch: {i+1}, Loss: {total_loss}')

In [27]:
# Import the following classes to create a combined dataset of X and y
from torch.utils.data import TensorDataset, DataLoader

# Create a training dataset
dataset_train = TensorDataset(X_train, y_train)
# Declare the batchsize and shuffle the data
dataloader_train = DataLoader(dataset_train, batch_size=10, shuffle=True)

# Traing the model using the train dataset
train_model(model, dataloader_train, optimizer, criterion, EPOCHS)

# Print out the loss of each epoch

Epoch: 1, Loss: 0.925717948935926
Epoch: 2, Loss: 1.006107903085649
Epoch: 3, Loss: 0.901418280787766
Epoch: 4, Loss: 1.079348498955369
Epoch: 5, Loss: 0.8502291981130838
Epoch: 6, Loss: 0.8308988343924284
Epoch: 7, Loss: 0.8804703135974705
Epoch: 8, Loss: 1.1590663064271212
Epoch: 9, Loss: 0.8411548747681081
Epoch: 10, Loss: 0.7079321211203933
Epoch: 11, Loss: 0.7225296390242875
Epoch: 12, Loss: 0.7773460848256946
Epoch: 13, Loss: 1.5963483941741288
Epoch: 14, Loss: 1.3809624953428283
Epoch: 15, Loss: 0.9175403108820319
Epoch: 16, Loss: 0.7848542057909071
Epoch: 17, Loss: 0.6354201603680849
Epoch: 18, Loss: 0.6804710766300559
Epoch: 19, Loss: 0.8124001547694206
Epoch: 20, Loss: 0.9377169851213694


In [28]:
# Creat a testing dataset
dataset_test = TensorDataset(X_test, y_test)
dataloader_test = DataLoader(dataset_test, batch_size=10, shuffle=True)

In [29]:
from sklearn.metrics import accuracy_score

In [30]:
# Create a testing function which returns total_loss of the model
def test_model(model, data):
  # Set the model to evaluation mode
  model.eval()
  # List for all predicted values
  all_preds = []
  # List for all actual values
  all_labels = []
  # No gradiant calculation
  with torch.no_grad():
    for X, y in data:
      # Predicted values for each X_test data
      pred_val = model(X)
      _, predicted = torch.max(pred_val, 1)
      all_preds.append(predicted) # Append the predicted values
      all_labels.append(y) # Append the actual values
    predicted = torch.cat(all_preds, dim=0) # Reshape it into 1 D array
    y = torch.cat(all_labels, dim=0) # Reshape it into 1 D array

    accuracy = accuracy_score(predicted, y) # Calculate the accuracy score
    return accuracy

In [31]:
# Test the accuracy of the model, using the testing dataset
test_accuracy = test_model(model, dataloader_test)
print(f'Test Accuracy: {test_accuracy}')

Test Accuracy: 0.9666666666666667
